In [23]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import statsmodels.formula.api as smf
import warnings
warnings.filterwarnings('ignore')
def variation_value(data,model,yname):
    data['yhat']=model.predict(data)
    SST=((data[yname]-data[yname].mean())**2).sum()
    SSR=((data['yhat']-data[yname].mean())**2).sum()
    SSE=((data[yname]-data['yhat'])**2).sum()
    return SST,SSR,SSE
def adjustedMetric(data,model,model_k,yname):
    SST,SSR,SSE=variation_value(data,model,yname)
    r2=SSR/SST
    adjustR2=1-(1-r2)*(data.shape[0]-1)/(data.shape[0]-model_k-1)
    RMSE=(SSE/(data.shape[0]-model_k-1))**0.5
    return adjustR2,RMSE
def assessTable(testdata,traindata,model,model_k,yname):
    r2test,RMSEtest=adjustedMetric(testdata,model,model_k,yname)
    r2train,RMSEtrain=adjustedMetric(traindata,model,model_k,yname)
    assessment=pd.DataFrame(index=['R2','RMSE'],
                        columns=['Train','Test'])
    assessment['Train']=[r2train,RMSEtrain]
    assessment['Test']=[r2test,RMSEtest]
    return assessment

## Get Full Model

We load data for module building .Different from previous sections, we will keep price process of sp500 for evaluation using practical standands

In [4]:
# read data into a DataFrame
data = pd.read_csv('data/indicepanel.csv',index_col=0)
data.head()
#raw data OF CLOSE PRICE

,nikkei,aord,nyse,dax,ftse,hangseng,sp500,djia
2013-09-06,13860.81,5144.0,9420.3480,8234.98,6547.33,22621.22,1655.08,14937.48
2013-09-09,14205.23,5179.4,9420.3480,8234.98,6530.74,22750.65,1655.08,14937.48
2013-09-10,14423.36,5198.9,9539.9320,8276.32,6583.99,22976.65,1671.71,15063.12
2013-09-11,14425.07,5230.6,9620.7100,8446.54,6588.43,22937.14,1683.99,15191.06
2013-09-12,14387.27,5238.2,9655.3774,8495.73,6588.98,22953.72,1689.13,15326.60


First we transform all price into log Return 

In [5]:
ReturnPanel=pd.DataFrame() # calculate log return
ReturnPanel['Price']=data['sp500']
ReturnPanel['sp500']=np.log(data['sp500'])-np.log(data['sp500'].shift(1)) # Response 
ReturnPanel['nikkei']=np.log(data['nikkei'])-np.log(data['nikkei'].shift(1))
ReturnPanel['aord']=np.log(data['aord'])-np.log(data['aord'].shift(1))
ReturnPanel['nyse']=np.log(data['nyse'])-np.log(data['nyse'].shift(1))
ReturnPanel['dax']=np.log(data['dax'])-np.log(data['dax'].shift(1))
ReturnPanel['ftse']=np.log(data['ftse'])-np.log(data['ftse'].shift(1))
ReturnPanel['hangseng']=np.log(data['hangseng'])-np.log(data['hangseng'].shift(1))
ReturnPanel['djia']=np.log(data['djia'])-np.log(data['djia'].shift(1))
ReturnPanel=ReturnPanel.dropna(axis=0)
ReturnPanel.head()

,Price,sp500,nikkei,aord,nyse,dax,ftse,hangseng,djia
2013-09-09,1655.08,0.000000,0.024545,0.006858,0.000000,0.000000,-0.002537,0.005705,0.000000
2013-09-10,1671.71,0.009998,0.015239,0.003758,0.012614,0.005007,0.008121,0.009885,0.008376
2013-09-11,1683.99,0.007319,0.000119,0.006079,0.008432,0.020358,0.000674,-0.001721,0.008458
2013-09-12,1689.13,0.003048,-0.002624,0.001452,0.003597,0.005807,0.000083,0.000723,0.008883
2013-09-13,1683.42,-0.003386,0.001209,-0.004496,-0.004990,-0.000204,-0.000786,-0.001676,-0.001695


In [6]:
dataMatrix=pd.DataFrame()
dataMatrix['Price']=ReturnPanel['Price']
dataMatrix['sp500']=ReturnPanel['sp500']# Y
dataMatrix['sp500_lag']=ReturnPanel['sp500'].shift(1)
dataMatrix['nikkei']=ReturnPanel['nikkei']
dataMatrix['nikkei_lag']=ReturnPanel['nikkei'].shift(1)
dataMatrix['aord']=ReturnPanel['aord']
dataMatrix['aord_lag']=ReturnPanel['aord'].shift(1)
dataMatrix['hangseng']=ReturnPanel['hangseng']
dataMatrix['hangseng_lag']=ReturnPanel['hangseng'].shift(1)
dataMatrix['nyse_lag']=ReturnPanel['nyse'].shift(1)
dataMatrix['dax_lag']=ReturnPanel['dax'].shift(1)
dataMatrix['ftse_lag']=ReturnPanel['ftse'].shift(1)
dataMatrix['djia_lag']=ReturnPanel['djia'].shift(1)
dataMatrix=dataMatrix.dropna(axis=0)# must-do step in order to build model 
dataMatrix.index=pd.to_datetime(dataMatrix.index)
print(type(dataMatrix.index[0]))
dataMatrix.head()

<class 'pandas._libs.tslib.Timestamp'>


,Price,sp500,sp500_lag,nikkei,nikkei_lag,aord,aord_lag,hangseng,hangseng_lag,nyse_lag,dax_lag,ftse_lag,djia_lag
2013-09-10,1671.71,0.009998,0.000000,0.015239,0.024545,0.003758,0.006858,0.009885,0.005705,0.000000,0.000000,-0.002537,0.000000
2013-09-11,1683.99,0.007319,0.009998,0.000119,0.015239,0.006079,0.003758,-0.001721,0.009885,0.012614,0.005007,0.008121,0.008376
2013-09-12,1689.13,0.003048,0.007319,-0.002624,0.000119,0.001452,0.006079,0.000723,-0.001721,0.008432,0.020358,0.000674,0.008458
2013-09-13,1683.42,-0.003386,0.003048,0.001209,-0.002624,-0.004496,0.001452,-0.001676,0.000723,0.003597,0.005807,0.000083,0.008883
2013-09-17,1697.60,0.008388,-0.003386,-0.006477,0.001209,0.005832,-0.004496,0.011508,-0.001676,-0.004990,-0.000204,-0.000786,-0.001695


The first column is close price of 'sp500'. The second column is  reponse and the rest columns are predictors. We split the data into train and test. 

In [7]:
print(dataMatrix.shape)
# avoid overfitting 
Train=dataMatrix.iloc[:780,:]
Test=dataMatrix.iloc[780:,:]

(974, 13)


## Feature Selection


Variable selection is intended to select the “best” subset of predictors.

- We want to explain the data in the simplest way — redundant predictors should be removed. The principle of Occam’s Razor states that among several plausible explanations for a phenomenon, the simplest is best. Applied to regression analysis, this implies that the smallest model that fits the data is best.

- Unnecessary predictors will add noise to the estimation of other quantities that we are interested in. 

- Collinearity is caused by having too many variables trying to do the same job.

**R-squared will always increase as you add more features to the model**, even if they are unrelated to the response. Thus, selecting the model with the highest R-squared is not a reliable approach for choosing the best linear model.

There is alternative to R-squared called **adjusted R-squared** that penalizes model complexity .

Last quesiton is how to try different models? we have 7 features, how many models should we try? totally $2^7$  models.

There are two main kinds of procedures
- Stepwise procedure
- Criterion-based procedures 




## Stepwise procedure
There are major stepwise procedures: backward elimination and forward selection algorithms.


#### Backward elimination


For backward elimination, we can use p-value approach or adjusted rsquare standard. 

Using p-value: 
- Start with all the predictors in the model
- Remove the predictor with highest p-value greater than significance level. 
- Refit the model and goto 2
- Stop when all p-values are less than significance level.

The significance level here  is alsocalled the “p-to-remove” and does not have to be 5%. If prediction performance is the goal, then a 15-20% cut-off may work best. 

Using adjusted rsquare: 
- Start with all the predictors in the model
- Remove the predictor whose reduced model has  highest adjusted  rquare. 
- Refit the model and goto 2
- Stop when all reduced models' adjusted rsquare is less than the best model in last step.   are less than significance level.



### Select Best Model Using Backward Elimination

We start with full model of 7 predictors. 

In [10]:
full= smf.ols(formula='sp500~nikkei+nikkei_lag+aord+aord_lag+ftse_lag+hangseng+hangseng_lag ', data=Train).fit()

We have 7 reduced model in the first step

In [11]:
del1= smf.ols(formula='sp500~nikkei_lag+aord+aord_lag+ftse_lag+hangseng+hangseng_lag ', data=Train).fit()
del2= smf.ols(formula='sp500~nikkei+aord+aord_lag+ftse_lag+hangseng+hangseng_lag ', data=Train).fit()
del3= smf.ols(formula='sp500~nikkei+nikkei_lag+aord_lag+ftse_lag+hangseng+hangseng_lag ', data=Train).fit()
del4= smf.ols(formula='sp500~nikkei+nikkei_lag+aord+ftse_lag+hangseng+hangseng_lag ', data=Train).fit()
del5= smf.ols(formula='sp500~nikkei+nikkei_lag+aord+aord_lag+hangseng+hangseng_lag ', data=Train).fit()
del6= smf.ols(formula='sp500~nikkei+nikkei_lag+aord+aord_lag+ftse_lag+hangseng_lag ', data=Train).fit()
del7= smf.ols(formula='sp500~nikkei+nikkei_lag+aord+aord_lag+ftse_lag+hangseng ', data=Train).fit()

where 'del'+'i' means , this model is achieved by deleting ith predictor in the full model.  We will prints adjusted rsquare for all reduced models in this step and full model. 

In [14]:
print(del1.rsquared_adj,del2.rsquared_adj,del3.rsquared_adj,del4.rsquared_adj)

0.227818419213 0.252806975054 0.236619291307 0.263503279036


In [13]:
print(del5.rsquared_adj,del6.rsquared_adj,del7.rsquared_adj,full.rsquared_adj)

0.24316755416 0.263040906667 0.260959095858 0.262819656417


We know that del4 is better than full model. Now we start from del4 to get 6 reduced models by cutting predictors fro del4. 

In [17]:
del41= smf.ols(formula='sp500~nikkei_lag+aord+ftse_lag+hangseng+hangseng_lag ', data=Train).fit()
del42= smf.ols(formula='sp500~nikkei+aord+ftse_lag+hangseng+hangseng_lag ', data=Train).fit()
del43= smf.ols(formula='sp500~nikkei+nikkei_lag+ftse_lag+hangseng+hangseng_lag ', data=Train).fit()
del44= smf.ols(formula='sp500~nikkei+nikkei_lag+aord+hangseng+hangseng_lag ', data=Train).fit()
del45= smf.ols(formula='sp500~nikkei+nikkei_lag+aord+ftse_lag+hangseng_lag ', data=Train).fit()
del46= smf.ols(formula='sp500~nikkei+nikkei_lag+aord+ftse_lag+hangseng ', data=Train).fit()

In [20]:
print(del41.rsquared_adj,del42.rsquared_adj,del43.rsquared_adj,del44.rsquared_adj)
print(del45.rsquared_adj,del46.rsquared_adj,del4.rsquared_adj)

0.227969775216 0.251198272685 0.23754460483 0.242497814785
0.263713395463 0.260623643976 0.263503279036


del45 is the best. We will start from del45 to get 5 reduced models. 

In [21]:
del451= smf.ols(formula='sp500~nikkei_lag+aord+ftse_lag+hangseng_lag ', data=Train).fit()
del452= smf.ols(formula='sp500~nikkei+aord+ftse_lag+hangseng_lag ', data=Train).fit()
del453= smf.ols(formula='sp500~nikkei+nikkei_lag+ftse_lag+hangseng_lag ', data=Train).fit()
del454= smf.ols(formula='sp500~nikkei+nikkei_lag+aord+hangseng_lag ', data=Train).fit()
del455= smf.ols(formula='sp500~nikkei+nikkei_lag+aord+ftse_lag', data=Train).fit()

In [22]:
print(del451.rsquared_adj,del452.rsquared_adj,del453.rsquared_adj,del454.rsquared_adj)
print(del455.rsquared_adj,del45.rsquared_adj)

0.220368891022 0.250920023226 0.229698023549 0.242265696015
0.261089560191 0.263713395463


The reduced modes in this step is not better than the model del45 in previous step. Hence the best model is del45

#### Evaluation using statistical standard

In [25]:
assessTable(Test,Train,full,7,'sp500')

,Train,Test
R2,0.262820,0.337431
RMSE,0.007672,0.003834


In [24]:
assessTable(Test,Train,del45,5,'sp500')

,Train,Test
R2,0.263713,0.355066
RMSE,0.007668,0.003855


We can find that the best model performances are better than those of full model.  

#### Evaluation using practical standards 

We need to get wealth process using different signals "FullSignal" and "BestSignal" given by different models "full" and "del45".

In [26]:
Trade_Train=pd.DataFrame()
Trade_Train['Price']=Train['Price']
Trade_Train['FullSignal']=full.predict(Train)
Trade_Train['BestSignal']=del45.predict(Train)
Trade_Train.head()

,Price,FullSignal,BestSignal
2013-09-10,1671.71,0.005535,0.005421
2013-09-11,1683.99,0.004513,0.004689
2013-09-12,1689.13,0.000366,0.000209
2013-09-13,1683.42,-0.000686,-0.000733
2013-09-17,1697.60,0.000670,0.000485


In [27]:
Trade_Test=pd.DataFrame()
Trade_Test['Price']=Test['Price']
Trade_Test['FullSignal']=full.predict(Test)
Trade_Test['BestSignal']=del45.predict(Test)
Trade_Test.head()

,Price,FullSignal,BestSignal
2016-11-22,2198.18,0.004084,0.003943
2016-11-24,2204.72,0.005026,0.005126
2016-11-25,2204.72,0.003296,0.003023
2016-11-28,2204.72,-0.000457,-0.000719
2016-11-29,2201.72,-0.000705,-0.000495


We will generate FullWealth and BestWealth using two different signals. 

In [29]:
Trade_Train['FullOrder']=[1 if sig>0 else -1 for sig in Trade_Train['FullSignal']  ]
Trade_Train['BestOrder']=[1 if sig>0 else -1 for sig in Trade_Train['BestSignal']  ]
Trade_Train['Price_change']=Trade_Train['Price']-Trade_Train['Price'].shift(1)
Trade_Train['FullProfit']=Trade_Train['Price_change']*Trade_Train['FullOrder']
Trade_Train['BestProfit']=Trade_Train['Price_change']*Trade_Train['BestOrder']
print(Trade_Train['FullProfit'].sum(),Trade_Train['BestProfit'].sum())
Trade_Train['FullWealth']=Trade_Train['FullProfit'].cumsum()+Trade_Train['Price'][0]
Trade_Train['BestWealth']=Trade_Train['BestProfit'].cumsum()+Trade_Train['Price'][0]


5111.91 5146.969999999999


In [31]:
Trade_Test['FullOrder']=[1 if sig>0 else -1 for sig in Trade_Test['FullSignal']  ]
Trade_Test['BestOrder']=[1 if sig>0 else -1 for sig in Trade_Test['BestSignal']  ]
Trade_Test['Price_change']=Trade_Test['Price']-Trade_Test['Price'].shift(1)
Trade_Test['FullProfit']=Trade_Test['Price_change']*Trade_Test['FullOrder']
Trade_Test['BestProfit']=Trade_Test['Price_change']*Trade_Test['BestOrder']
print(Trade_Test['FullProfit'].sum(),Trade_Test['BestProfit'].sum())
Trade_Test['FullWealth']=Trade_Test['FullProfit'].cumsum()+Trade_Test['Price'][0]
Trade_Test['BestWealth']=Trade_Test['BestProfit'].cumsum()+Trade_Test['Price'][0]

665.5699999999979 677.0299999999988


Next we compare drawdown in both train and test

In [34]:
# COMPUTE  maximum process: highest wealth ever achieved so far 
Trade_Test['FullPeak']=[Trade_Test.loc[:any_index,'FullWealth'].max() 
                    for any_index in Trade_Test.index ]
Trade_Test['BestPeak']=[Trade_Test.loc[:any_index,'BestWealth'].max() 
                    for any_index in Trade_Test.index ]
Trade_Test['FullDrawdown']=(Trade_Test['FullPeak']-Trade_Test['FullWealth'])/Trade_Test['FullPeak']
Trade_Test['BestDrawdown']=(Trade_Test['BestPeak']-Trade_Test['BestWealth'])/Trade_Test['BestPeak']

print("Maximum drawdown rate in test is", "Full: ", Trade_Test['FullDrawdown'].max()," Best: ",Trade_Test['BestDrawdown'].max())


Maximum drawdown rate in test is Full:  0.0118873781314  Best:  0.0118873781314


In [35]:
# COMPUTE  maximum process: highest wealth ever achieved so far 
Trade_Train['FullPeak']=[Trade_Train.loc[:any_index,'FullWealth'].max() 
                    for any_index in Trade_Train.index ]
Trade_Train['BestPeak']=[Trade_Train.loc[:any_index,'BestWealth'].max() 
                    for any_index in Trade_Train.index ]
Trade_Train['FullDrawdown']=(Trade_Train['FullPeak']-Trade_Train['FullWealth'])/Trade_Train['FullPeak']
Trade_Train['BestDrawdown']=(Trade_Train['BestPeak']-Trade_Train['BestWealth'])/Trade_Train['BestPeak']

print("Maximum drawdown rate in Train is", "Full: ", Trade_Train['FullDrawdown'].max()," Best: ",Trade_Train['BestDrawdown'].max())


Maximum drawdown rate in Train is Full:  0.0138127921266  Best:  0.0138127921266


Maximum drawdown are the same for two signals in both train and test. Next we will compare sharpe ratios of two models in both train and test

In [39]:
#daily return process of wealth
Trade_Train['FullReturn']=(Trade_Train['FullWealth']
                      -Trade_Train['FullWealth'].shift(1))/Trade_Train['FullWealth'].shift(1)
Trade_Train['BestReturn']=(Trade_Train['BestWealth']
                      -Trade_Train['BestWealth'].shift(1))/Trade_Train['BestWealth'].shift(1)
dailybest=Trade_Train['BestReturn'].dropna()
dailyfull=Trade_Train['FullReturn'].dropna()
print("yearly sharpe ratio for the full mdoel is  ", (220**0.5)*dailyfull.mean()/dailyfull.std(ddof=1))
print("yearly sharpe ratio for the best mdoel is  ", (220**0.5)*dailybest.mean()/dailybest.std(ddof=1))


yearly sharpe ratio for the full mdoel is   6.119066805280244
yearly sharpe ratio for the best mdoel is   6.149920704217581


In [40]:

Trade_Test['FullReturn']=(Trade_Test['FullWealth']
                      -Trade_Test['FullWealth'].shift(1))/Trade_Test['FullWealth'].shift(1)
Trade_Test['BestReturn']=(Trade_Test['BestWealth']
                      -Trade_Test['BestWealth'].shift(1))/Trade_Test['BestWealth'].shift(1)
dailybest=Trade_Test['BestReturn'].dropna()
dailyfull=Trade_Test['FullReturn'].dropna()
print("yearly sharpe ratio for the full mdoel is  ", (220**0.5)*dailyfull.mean()/dailyfull.std(ddof=1))
print("yearly sharpe ratio for the best mdoel is  ", (220**0.5)*dailybest.mean()/dailybest.std(ddof=1))

yearly sharpe ratio for the full mdoel is   5.2322513219192865
yearly sharpe ratio for the best mdoel is   5.3581849376915365


In both train and test set, sharpe ratio increases in the best model. 

### Forward Selection
We also can use p-value approach and adjusted Rsquare

Using p-value:
- Start with no variables in the model.
- For all predictors not in the model, check their p-value if they are added to the model. Choose the one with lowest p-value less than significance level .
- Continue until no new predictors can be added.

The significance level here  is alsocalled the “p-to-add” 

Using adjusted Rsquare:
- Start with no variables in the model (base model).
- Add the predictor to base model (or best model in last step) whose adjusted rsquare is highest
- Continue until no new predictors can be added (Adding new predictor does not increase adjusted rsquare)
 

## Criterion-based Procedures

If there are k potential predictors, then there are $2^k$ possible models. We fit all these models and choose the best one according to some criterion. Here are some oftern used criterion.

- The Akaike Information Criterion (AIC) and the Bayes Information Criterion (BIC) are some other commonly used criteria. In general,
$$
AIC=2k + n Log(\frac{SSE}{n})
$$
where
$$
BIC=kLog(n)+n Log(\frac{SSE}{n})
$$

We want to minimize AIC or BIC. Larger models will fit better and so have smaller RSS but use more parameters. Thus the best choice of model will balance fit with model size. BIC penalizes larger models more heavily and so will tend to prefer smaller models in comparison to AIC

- Adjusted $R^2$

    - The model out of $2^k$ different models who has highest adjusted rsquare is the best model. 

## Conclusion
Stepwise methods use a restricted search through the space of potential models. Criterion-based methods typically involve a wider search and compare models in a preferable manner. 

In practice, we may use correlation or other association measure to select a small pool of predictors (say 100-500) from a larger pool of candidates (>2000). Then we use backward elimination or criterion based models to select the best model from this small pool. 

feature selection or model selection can significantly improve the performance of the model in both consistency and effectiveness. 